In [1]:
import hgfp
import torch
import dgl
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from matplotlib import pyplot as plt

RDKit WARNING: [12:05:35] Enabling RDKit 2019.09.3 jupyter extensions


SyntaxError: invalid syntax (hierachical_message_passing.py, line 64)

In [2]:
element_to_idx = {b'BR': 0, b'C': 1, b'C2': 2, b'CA': 3, b'CB': 4, b'CC': 5, b'CJ': 6, b'CL': 7, b'CM': 8, b'CP': 9, b'CR': 10, b'CT': 11, b'CW': 12, b'Cstar': 13, b'F': 14, b'H': 15, b'H1': 16, b'H2': 17, b'H3': 18, b'H4': 19, b'H5': 20, b'HA': 21, b'HC': 22, b'HO': 23, b'HP': 24, b'HX': 25, b'I': 26, b'N': 27, b'N2': 28, b'N3': 29, b'NA': 30, b'NB': 31, b'NC': 32, b'NL': 33, b'Nstar': 34, b'Nu': 35, b'O': 36, b'O2': 37, b'OH': 38, b'OS': 39, b'Ou': 40, b'P': 41, b'S': 42, b'SO': 43, b'Su': 44}

In [3]:
idx_to_element = {v: k.decode("utf-8") for k, v in element_to_idx.items()}

In [4]:
ds = list(hgfp.data.parm_at_Frosst.df.batched(num=1000, batch_size=16, use_fp=False))

/Users/wangy1/anaconda3/envs/env1/lib/python3.7/site-packages/dgl/base.py:25: UserWarning: Currently adjacency_matrix() returns a matrix with destination as rows by default.  In 0.5 the result will have source as rows (i.e. transpose=True)
  warnings.warn(msg, warn_type)


In [5]:
print(len(ds))

62


In [6]:
ds_tr, ds_vl, ds_te = hgfp.data.utils.split(ds, 10, 10)

In [7]:
class GN(torch.nn.Module):
    def __init__(self, model, kwargs):
        super(GN, self).__init__()
        self.gn = model(64, 64, **kwargs)
        
    def forward(self, g, x):
        g_sub = dgl.to_homo(
            g.edge_type_subgraph(['atom_neighbors_atom']))
        x = self.gn(g_sub, x)
        return x        

class Classifier(torch.nn.Module):
    def __init__(self, in_dim=128, out_dim=256, n_classes=45):
        super(Classifier, self).__init__()
        self.d = torch.nn.Linear(in_dim, out_dim)
        self.c = torch.nn.Linear(out_dim, n_classes)
        
    def forward(self, x):
        y_hat = self.c(
                torch.nn.functional.tanh(
                    self.d(
                        x)))
        
        return y_hat
        
class Net(torch.nn.Module):
    def __init__(self, model, kwargs):
        super(Net, self).__init__()
        self.f_in = torch.nn.Sequential(
            torch.nn.Linear(100, 64),
            torch.nn.Tanh())
        
        self.gn0 = GN(model, kwargs)
        self.gn1 = GN(model, kwargs)
        self.gn2 = GN(model, kwargs)
        
        self.c = Classifier(64, 64, 45)
        
    def forward(self, g):
        x = g.nodes['atom'].data['h0']
        x = self.f_in(x)
        x = self.gn0(g, x)
        x = torch.nn.functional.tanh(x)
        x = self.gn1(g, x)
        x = torch.nn.functional.tanh(x)
        x = self.gn2(g, x)
        x = self.c(x)
        return x
        

In [8]:
from dgl.nn import pytorch as dgl_pytorch
net = Net(dgl_pytorch.conv.SAGEConv,  {'aggregator_type': 'mean'})

In [9]:
# opt = torch.optim.Adam(list(net.parameters()) + list(classifier.parameters()), 1e-3)
opt = torch.optim.Adam(list(net.parameters()), 1e-3)

In [10]:
loss_fn = torch.nn.CrossEntropyLoss()

In [11]:
for _ in range(100):
    for g, y in ds_tr:
        opt.zero_grad()
        # y_hat = classifier(net(g, return_graph=True))
        y_hat = net(g)
        loss = loss_fn(y_hat, torch.where(torch.gt(y, 0))[1])
        print(loss)
        loss.backward()
        opt.step()

/Users/wangy1/anaconda3/envs/env1/lib/python3.7/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor(3.9037, grad_fn=<NllLossBackward>)
tensor(3.7834, grad_fn=<NllLossBackward>)
tensor(3.6582, grad_fn=<NllLossBackward>)
tensor(3.6150, grad_fn=<NllLossBackward>)
tensor(3.5480, grad_fn=<NllLossBackward>)
tensor(3.4715, grad_fn=<NllLossBackward>)
tensor(3.3929, grad_fn=<NllLossBackward>)
tensor(3.2598, grad_fn=<NllLossBackward>)
tensor(3.3902, grad_fn=<NllLossBackward>)
tensor(3.2773, grad_fn=<NllLossBackward>)
tensor(3.2124, grad_fn=<NllLossBackward>)
tensor(3.1625, grad_fn=<NllLossBackward>)
tensor(3.1057, grad_fn=<NllLossBackward>)
tensor(3.1575, grad_fn=<NllLossBackward>)
tensor(3.1295, grad_fn=<NllLossBackward>)
tensor(3.0557, grad_fn=<NllLossBackward>)
tensor(2.8966, grad_fn=<NllLossBackward>)
tensor(3.0219, grad_fn=<NllLossBackward>)
tensor(2.9823, grad_fn=<NllLossBackward>)
tensor(2.9135, grad_fn=<NllLossBackward>)
tensor(2.8130, grad_fn=<NllLossBackward>)
tensor(2.7999, grad_fn=<NllLossBackward>)
tensor(2.7781, grad_fn=<NllLossBackward>)
tensor(2.7222, grad_fn=<NllLossBac

tensor(0.7421, grad_fn=<NllLossBackward>)
tensor(0.6531, grad_fn=<NllLossBackward>)
tensor(0.7016, grad_fn=<NllLossBackward>)
tensor(0.7851, grad_fn=<NllLossBackward>)
tensor(0.6769, grad_fn=<NllLossBackward>)
tensor(0.7274, grad_fn=<NllLossBackward>)
tensor(0.6235, grad_fn=<NllLossBackward>)
tensor(0.6829, grad_fn=<NllLossBackward>)
tensor(0.7262, grad_fn=<NllLossBackward>)
tensor(0.7680, grad_fn=<NllLossBackward>)
tensor(0.7104, grad_fn=<NllLossBackward>)
tensor(0.7166, grad_fn=<NllLossBackward>)
tensor(0.7853, grad_fn=<NllLossBackward>)
tensor(0.8317, grad_fn=<NllLossBackward>)
tensor(0.8121, grad_fn=<NllLossBackward>)
tensor(0.8121, grad_fn=<NllLossBackward>)
tensor(0.8025, grad_fn=<NllLossBackward>)
tensor(0.7322, grad_fn=<NllLossBackward>)
tensor(0.6882, grad_fn=<NllLossBackward>)
tensor(0.6392, grad_fn=<NllLossBackward>)
tensor(0.7540, grad_fn=<NllLossBackward>)
tensor(0.7162, grad_fn=<NllLossBackward>)
tensor(0.5946, grad_fn=<NllLossBackward>)
tensor(0.6132, grad_fn=<NllLossBac

tensor(0.6179, grad_fn=<NllLossBackward>)
tensor(0.5671, grad_fn=<NllLossBackward>)
tensor(0.5748, grad_fn=<NllLossBackward>)
tensor(0.5697, grad_fn=<NllLossBackward>)
tensor(0.5936, grad_fn=<NllLossBackward>)
tensor(0.5773, grad_fn=<NllLossBackward>)
tensor(0.5070, grad_fn=<NllLossBackward>)
tensor(0.5478, grad_fn=<NllLossBackward>)
tensor(0.6430, grad_fn=<NllLossBackward>)
tensor(0.5485, grad_fn=<NllLossBackward>)
tensor(0.5746, grad_fn=<NllLossBackward>)
tensor(0.5132, grad_fn=<NllLossBackward>)
tensor(0.5290, grad_fn=<NllLossBackward>)
tensor(0.5747, grad_fn=<NllLossBackward>)
tensor(0.5925, grad_fn=<NllLossBackward>)
tensor(0.5680, grad_fn=<NllLossBackward>)
tensor(0.5554, grad_fn=<NllLossBackward>)
tensor(0.6347, grad_fn=<NllLossBackward>)
tensor(0.6543, grad_fn=<NllLossBackward>)
tensor(0.6647, grad_fn=<NllLossBackward>)
tensor(0.6843, grad_fn=<NllLossBackward>)
tensor(0.6293, grad_fn=<NllLossBackward>)
tensor(0.5993, grad_fn=<NllLossBackward>)
tensor(0.5393, grad_fn=<NllLossBac

tensor(0.5467, grad_fn=<NllLossBackward>)
tensor(0.4967, grad_fn=<NllLossBackward>)
tensor(0.4960, grad_fn=<NllLossBackward>)
tensor(0.4952, grad_fn=<NllLossBackward>)
tensor(0.5113, grad_fn=<NllLossBackward>)
tensor(0.4985, grad_fn=<NllLossBackward>)
tensor(0.4279, grad_fn=<NllLossBackward>)
tensor(0.4882, grad_fn=<NllLossBackward>)
tensor(0.5717, grad_fn=<NllLossBackward>)
tensor(0.4838, grad_fn=<NllLossBackward>)
tensor(0.5042, grad_fn=<NllLossBackward>)
tensor(0.4672, grad_fn=<NllLossBackward>)
tensor(0.4598, grad_fn=<NllLossBackward>)
tensor(0.5077, grad_fn=<NllLossBackward>)
tensor(0.5090, grad_fn=<NllLossBackward>)
tensor(0.5037, grad_fn=<NllLossBackward>)
tensor(0.4850, grad_fn=<NllLossBackward>)
tensor(0.5605, grad_fn=<NllLossBackward>)
tensor(0.5817, grad_fn=<NllLossBackward>)
tensor(0.5972, grad_fn=<NllLossBackward>)
tensor(0.6234, grad_fn=<NllLossBackward>)
tensor(0.5490, grad_fn=<NllLossBackward>)
tensor(0.5461, grad_fn=<NllLossBackward>)
tensor(0.4688, grad_fn=<NllLossBac

...

tensor(0.2537, grad_fn=<NllLossBackward>)
tensor(0.1997, grad_fn=<NllLossBackward>)
tensor(0.2860, grad_fn=<NllLossBackward>)
tensor(0.2356, grad_fn=<NllLossBackward>)
tensor(0.3264, grad_fn=<NllLossBackward>)
tensor(0.2547, grad_fn=<NllLossBackward>)
tensor(0.3117, grad_fn=<NllLossBackward>)
tensor(0.3196, grad_fn=<NllLossBackward>)
tensor(0.3590, grad_fn=<NllLossBackward>)
tensor(0.3531, grad_fn=<NllLossBackward>)
tensor(0.3028, grad_fn=<NllLossBackward>)
tensor(0.3568, grad_fn=<NllLossBackward>)
tensor(0.2262, grad_fn=<NllLossBackward>)
tensor(0.2535, grad_fn=<NllLossBackward>)
tensor(0.2444, grad_fn=<NllLossBackward>)
tensor(0.2635, grad_fn=<NllLossBackward>)
tensor(0.1588, grad_fn=<NllLossBackward>)
tensor(0.2510, grad_fn=<NllLossBackward>)
tensor(0.2077, grad_fn=<NllLossBackward>)
tensor(0.2537, grad_fn=<NllLossBackward>)
tensor(0.2660, grad_fn=<NllLossBackward>)
tensor(0.3351, grad_fn=<NllLossBackward>)
tensor(0.2463, grad_fn=<NllLossBackward>)
tensor(0.2082, grad_fn=<NllLossBac

tensor(0.3030, grad_fn=<NllLossBackward>)
tensor(0.2728, grad_fn=<NllLossBackward>)
tensor(0.2408, grad_fn=<NllLossBackward>)
tensor(0.2331, grad_fn=<NllLossBackward>)
tensor(0.2694, grad_fn=<NllLossBackward>)
tensor(0.2339, grad_fn=<NllLossBackward>)
tensor(0.2564, grad_fn=<NllLossBackward>)
tensor(0.2102, grad_fn=<NllLossBackward>)
tensor(0.2759, grad_fn=<NllLossBackward>)
tensor(0.2703, grad_fn=<NllLossBackward>)
tensor(0.2872, grad_fn=<NllLossBackward>)
tensor(0.2477, grad_fn=<NllLossBackward>)
tensor(0.2449, grad_fn=<NllLossBackward>)
tensor(0.1989, grad_fn=<NllLossBackward>)
tensor(0.2865, grad_fn=<NllLossBackward>)
tensor(0.2307, grad_fn=<NllLossBackward>)
tensor(0.3173, grad_fn=<NllLossBackward>)
tensor(0.2454, grad_fn=<NllLossBackward>)
tensor(0.2966, grad_fn=<NllLossBackward>)
tensor(0.2983, grad_fn=<NllLossBackward>)
tensor(0.3413, grad_fn=<NllLossBackward>)
tensor(0.3430, grad_fn=<NllLossBackward>)
tensor(0.2985, grad_fn=<NllLossBackward>)
tensor(0.3456, grad_fn=<NllLossBac

tensor(0.2318, grad_fn=<NllLossBackward>)
tensor(0.2572, grad_fn=<NllLossBackward>)
tensor(0.3162, grad_fn=<NllLossBackward>)
tensor(0.2416, grad_fn=<NllLossBackward>)
tensor(0.2044, grad_fn=<NllLossBackward>)
tensor(0.1524, grad_fn=<NllLossBackward>)
tensor(0.2390, grad_fn=<NllLossBackward>)
tensor(0.2347, grad_fn=<NllLossBackward>)
tensor(0.2420, grad_fn=<NllLossBackward>)
tensor(0.3795, grad_fn=<NllLossBackward>)
tensor(0.2530, grad_fn=<NllLossBackward>)
tensor(0.2946, grad_fn=<NllLossBackward>)
tensor(0.2666, grad_fn=<NllLossBackward>)
tensor(0.2349, grad_fn=<NllLossBackward>)
tensor(0.2304, grad_fn=<NllLossBackward>)
tensor(0.2680, grad_fn=<NllLossBackward>)
tensor(0.2330, grad_fn=<NllLossBackward>)
tensor(0.2508, grad_fn=<NllLossBackward>)
tensor(0.2034, grad_fn=<NllLossBackward>)
tensor(0.2689, grad_fn=<NllLossBackward>)
tensor(0.2554, grad_fn=<NllLossBackward>)
tensor(0.2773, grad_fn=<NllLossBackward>)
tensor(0.2396, grad_fn=<NllLossBackward>)
tensor(0.2342, grad_fn=<NllLossBac

tensor(0.2358, grad_fn=<NllLossBackward>)
tensor(0.1465, grad_fn=<NllLossBackward>)
tensor(0.2198, grad_fn=<NllLossBackward>)
tensor(0.1825, grad_fn=<NllLossBackward>)
tensor(0.2273, grad_fn=<NllLossBackward>)
tensor(0.2516, grad_fn=<NllLossBackward>)
tensor(0.3045, grad_fn=<NllLossBackward>)
tensor(0.2399, grad_fn=<NllLossBackward>)
tensor(0.1987, grad_fn=<NllLossBackward>)
tensor(0.1455, grad_fn=<NllLossBackward>)
tensor(0.2366, grad_fn=<NllLossBackward>)
tensor(0.2283, grad_fn=<NllLossBackward>)
tensor(0.2391, grad_fn=<NllLossBackward>)
tensor(0.3726, grad_fn=<NllLossBackward>)
tensor(0.2440, grad_fn=<NllLossBackward>)
tensor(0.2861, grad_fn=<NllLossBackward>)
tensor(0.2625, grad_fn=<NllLossBackward>)
tensor(0.2276, grad_fn=<NllLossBackward>)
tensor(0.2229, grad_fn=<NllLossBackward>)
tensor(0.2670, grad_fn=<NllLossBackward>)
tensor(0.2344, grad_fn=<NllLossBackward>)
tensor(0.2512, grad_fn=<NllLossBackward>)
tensor(0.2004, grad_fn=<NllLossBackward>)
tensor(0.2592, grad_fn=<NllLossBac

tensor(0.2126, grad_fn=<NllLossBackward>)
tensor(0.2275, grad_fn=<NllLossBackward>)
tensor(0.1437, grad_fn=<NllLossBackward>)
tensor(0.2126, grad_fn=<NllLossBackward>)
tensor(0.1791, grad_fn=<NllLossBackward>)
tensor(0.2255, grad_fn=<NllLossBackward>)
tensor(0.2494, grad_fn=<NllLossBackward>)
tensor(0.2977, grad_fn=<NllLossBackward>)
tensor(0.2397, grad_fn=<NllLossBackward>)
tensor(0.1939, grad_fn=<NllLossBackward>)
tensor(0.1388, grad_fn=<NllLossBackward>)
tensor(0.2296, grad_fn=<NllLossBackward>)
tensor(0.2202, grad_fn=<NllLossBackward>)
tensor(0.2334, grad_fn=<NllLossBackward>)
tensor(0.3613, grad_fn=<NllLossBackward>)
tensor(0.2373, grad_fn=<NllLossBackward>)
tensor(0.2821, grad_fn=<NllLossBackward>)
tensor(0.2626, grad_fn=<NllLossBackward>)
tensor(0.2235, grad_fn=<NllLossBackward>)
tensor(0.2154, grad_fn=<NllLossBackward>)
tensor(0.2644, grad_fn=<NllLossBackward>)
tensor(0.2316, grad_fn=<NllLossBackward>)
tensor(0.2494, grad_fn=<NllLossBackward>)
tensor(0.1971, grad_fn=<NllLossBac

In [12]:
net.eval()
from sklearn.metrics import confusion_matrix

y_hat = torch.cat([torch.argmax(net(g), dim=1) for g, y in ds_vl+ds_te], axis=0)
y = torch.cat([torch.argmax(y, dim=1) for g, y in ds_vl+ds_te], axis=0)

print(np.count_nonzero(y - y_hat) / y_hat.shape[0])

0.10292422177968766


/Users/wangy1/anaconda3/envs/env1/lib/python3.7/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [13]:
import pandas as pd
df_cm = pd.DataFrame(confusion_matrix(y, y_hat, labels=list(range(1, 46))),
    [v for k, v in idx_to_element.items()],
    [v for k, v in idx_to_element.items()])

In [14]:
count_matrix = df_cm.values
wrong_idxs = np.stack(np.where(np.greater(count_matrix, 0)), axis=1)
wrong_idxs = wrong_idxs[wrong_idxs[:, 0] != wrong_idxs[:, 1]]
wrong_count = np.array([count_matrix[idxs[0]][idxs[1]] for idxs in wrong_idxs])
wrong_count_argsort = np.flip(np.argsort(wrong_count))
for idx in wrong_count_argsort:
    print('%s -> %s : %s'%(
        idx_to_element[wrong_idxs[idx][0]],
        idx_to_element[wrong_idxs[idx][1]],
        wrong_count[idx]))

CL -> C2 : 65
NA -> NB : 45
C2 -> CB : 43
NB -> NA : 40
C2 -> CT : 40
CA -> C2 : 39
CB -> C2 : 34
CT -> C2 : 32
C2 -> CA : 31
C2 -> BR : 27
O -> Nu : 26
C2 -> CL : 26
C2 -> CP : 22
CP -> C2 : 20
CA -> CB : 19
N -> N3 : 19
OH -> Nu : 18
CM -> C2 : 17
BR -> C2 : 16
CA -> CT : 15
CL -> CR : 14
CB -> CA : 14
CT -> BR : 13
I -> N : 13
CL -> CT : 12
Nu -> O : 12
CT -> CA : 11
NL -> NA : 11
BR -> CP : 10
CP -> BR : 10
CA -> CP : 9
CC -> CR : 9
CB -> CT : 9
HO -> H : 8
CL -> CB : 8
H5 -> HA : 8
I -> NA : 8
N -> NL : 8
CT -> CB : 7
N -> I : 7
NL -> NB : 7
CT -> CP : 6
CR -> BR : 6
NL -> N : 6
N -> N2 : 6
NA -> N3 : 5
BR -> CR : 5
CM -> CB : 5
CP -> CT : 5
CP -> CW : 5
BR -> CB : 4
N2 -> NA : 4
C2 -> CR : 4
N -> NA : 4
NA -> I : 4
CM -> CA : 3
S -> SO : 3
CL -> C : 3
CA -> BR : 3
N2 -> NL : 3
CR -> C2 : 3
CR -> CT : 3
NL -> I : 3
N -> NB : 3
NB -> I : 3
BR -> CW : 3
HP -> HC : 3
CW -> BR : 3
BR -> CT : 3
C -> CR : 3
CW -> CT : 3
N3 -> N : 3
CB -> CP : 2
O -> OS : 2
CB -> BR : 2
CB -> CL : 2
CL -